In [5]:
import scarlink as sl
import scarlink.src.visualization as scv
from scarlink.src.read_model import read_model
import matplotlib.pyplot as plt
import glob
import h5py
import pandas
import numpy as np
import os
import seaborn
from scipy import stats

2024-05-11 21:35:14.750068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 21:35:14.868934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-11 21:35:14.868958: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-05-11 21:35:15.471006: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [7]:
os.chdir('../SCARlink_analysis/downloaded')
os.getcwd()

'/media/NaturalPopulationCohort/caipengfei/scCAT/SCARlink_analysis/downloaded'

In [9]:
### Download scripts from figshare
!( \
if [ ! -d ./scripts ]; then \
wget https://figshare.com/ndownloader/files/44297132?private_link=9b9e89ff3150aebb6d7a -O ./scripts.zip; \
unzip scripts.zip; rm scripts.zip; \
fi)

--2024-05-11 16:13:03--  https://figshare.com/ndownloader/files/44297132?private_link=9b9e89ff3150aebb6d7a
正在解析主机 figshare.com (figshare.com)... 34.248.40.228, 52.214.121.144, 2a05:d018:1f4:d003:e29a:4e34:ef7f:49e1, ...
正在连接 figshare.com (figshare.com)|34.248.40.228|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 403 Forbidden
2024-05-11 16:13:04 错误 403：Forbidden。

Archive:  scripts.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
note:  scripts.zip may be a plain executable, not an archive
unzip:  cannot find zipfile directory in one of scripts.zip or
        scripts.zip.zip, and cannot find scripts.zip.ZIP, period.


In [8]:
from scripts.compare_corrs import plot_compare_gsm_corr, plot_compare_dorc_corr
from scripts.compare_corrs import make_gsm_scarlink_table

In [9]:
def calc_fishers_p(r12, r13, r23, n):
    # adjust for numerical issues
    det_s = max(0, 1 - (r12**2 + r13**2 + r23**2) + 2*r12*r13*r23)

    t = (n-1)*(1+r23)
    t /= 2*((n-1)/(n-3))*det_s + (r12+r13)**2/4*math.pow(1-r23, 3)
    t = abs(r12-r13)*math.sqrt(t)
    pval = stats.t.sf(t, n-3)
    return pval

def get_gsm_gene(gsm, gene_names, rm, gene):
    # gsm: Gene score matrix with cell x gene score
    # gene_names: Names of genes in columns

    gene_gsm = np.ravel(gsm[gene_names['name'] == gene, :].todense())
    return gene_gsm[rm.test_ix]

def get_model_gene(rm, f, gene):
    # get SCARlink predictions for given gene
    w_mat = np.array(f['genes/' + gene][:])
    e = np.array([f['genes/' + gene].attrs['intercept']])
    params = [w_mat, e]
    train_alpha = float(f['genes/' + gene].attrs['alpha'])
    gex_train, gex_test = rm.get_gex_gene(gene)
    tile_gene_mat_train, tile_gene_mat_test = rm.gene_tile_matrix_scaled(gene,
                                                    normalization_factor='ReadsInTSS')
    model_custom = rm.build_model(tile_gene_mat_test.shape[1], train_alpha)
    model_custom.set_weights(params)
    pred_vals = np.ravel(model_custom(tile_gene_mat_test.todense(), training=False).numpy())
    return pred_vals

In [10]:
def make_scarlink_table_DiffTechs(dirname1, dirname2, out_prefix, check_hvg=True):
    # create table with Spearman correlations between predicted gene expression from SCARlink using different techniques

    dirname1 = dirname1 + '/' if dirname1[-1] != '/' else dirname1
    dirname2 = dirname2 + '/' if dirname2[-1] != '/' else dirname2

    if check_hvg:
        # Initially model was run on more genes. We are going to use two overlap genes
        if os.path.isfile(dirname1 + 'hvg.txt'):
            hvg1 = pandas.read_csv(dirname1 + 'hvg.txt', header=None)[0].values.tolist()
        if os.path.isfile(dirname2 + 'hvg.txt'):
            hvg2 = pandas.read_csv(dirname2 + 'hvg.txt', header=None)[0].values.tolist()
        hvg=[i for i in hvg1 if i in hvg2]

    dirname1 = dirname1 + "scarlink_out/"
    dirname2 = dirname2 + "scarlink_out/"
    filename = out_prefix + '_values.csv'

    if os.path.isfile(filename):
        df = pandas.read_csv(filename, sep='\t')
        return df


    coef_file_1 = dirname1 + 'coefficients_None.hd5'
    coef_file_2 = dirname2 + 'coefficients_None.hd5'
    
    # tbl = pandas.DataFrame(columns=['gene', 'model_test_corr'])
    model1_corrs = []
    model2_corrs = []
    
    better_method = []
    gene_name = []
    
    #model1_model2_corrs = []
    #method_corr = []
    
    f1 = h5py.File(coef_file_1, mode = 'r')
    f2 = h5py.File(coef_file_2, mode = 'r')
    
    f_genes1 = list(f1['genes/'].keys())
    f_genes2 = list(f2['genes/'].keys())
    f_genes=[i for i in f_genes1 if i in f_genes2]
    
    rm1 = read_model(dirname1, out_file_name=coef_file_1.split('/')[-1], read_only=True)
    rm2 = read_model(dirname2, out_file_name=coef_file_2.split('/')[-1], read_only=True)
        
    if check_hvg and hvg != []:
        f_genes = list(filter(lambda x: x in hvg, f_genes))

    for gene in f_genes:
        m1 = get_model_gene(rm1, f1, gene)
        _, obs1 = rm1.get_gex_gene(gene)
        obs1 = np.ravel(obs1.todense())
        m1_obs, _ = stats.spearmanr(m1, obs1)
        
        m2 = get_model_gene(rm2, f2, gene)
        _, obs2 = rm2.get_gex_gene(gene)
        obs2 = np.ravel(obs2.todense())
        m2_obs, _ = stats.spearmanr(m2, obs2)

        #m1_m2, _ = stats.spearmanr(m1, m2)
        
        model1_corrs.append(m1_obs)
        model2_corrs.append(m2_obs)
        
        #model1_model2_corrs.append(m1_m2)
        #fishers_corr = calc_fishers_p(m_obs, g_obs, m_g, rm.test_ix.shape[0])
        #method_corr.append(fishers_corr)
        
        gene_name.append(gene)
        better_method.append(1 if m1_obs > m2_obs else -1)
    f1.close()
    f2.close()
    
    df = pandas.DataFrame(columns=['method 1', 'method 2', 'gene'])
    df['method 1'] = model1_corrs
    df['method 2'] = model2_corrs
    df['gene'] = gene_name
    df['better_method_flag'] = better_method
    df.to_csv(filename, sep='\t', index=None)
    return df

In [11]:
def make_scarlink_table(dirname, check_hvg=True):
    # create table with Spearman correlations between predicted gene expression from SCARlink

    dirname = dirname + '/' if dirname[-1] != '/' else dirname

    if check_hvg:
        # Initially model was run on more genes. We are going to use two overlap genes
        if os.path.isfile(dirname + 'hvg.txt'):
            hvg = pandas.read_csv(dirname + 'hvg.txt', header=None)[0].values.tolist()
        else:
            hvg=[]

    dirname = dirname + "scarlink_out/"
    
    coef_file = dirname + 'coefficients_None.hd5'
    
    model_corrs = []
    gene_name = []
    
    f = h5py.File(coef_file, mode = 'r')
    
    f_genes = list(f['genes/'].keys())
    
    rm = read_model(dirname, out_file_name=coef_file.split('/')[-1], read_only=True)
      
    if check_hvg and hvg != []:
        f_genes = list(filter(lambda x: x in hvg, f_genes))

    for gene in f_genes:
        m = get_model_gene(rm, f, gene)
        _, obs = rm.get_gex_gene(gene)
        obs = np.ravel(obs.todense())
        m_obs, _ = stats.spearmanr(m, obs)
        
        model_corrs.append(m_obs)
        
        gene_name.append(gene)
        
    f.close()
    
    df = pandas.DataFrame(columns=['corr', 'gene'])
    df['corr'] = model_corrs
    df['gene'] = gene_name

    return df

In [ ]:
def plot_compare_gsm_corr(df, plot_title):
    
    print("Number of genes:", df.shape)
    min_val = np.nanmin(df[['method 1', 'method 2']].values)
    max_val = np.nanmax(df[['method 1', 'method 2']].values)

    pv = df['fishers_p'].values
    pv[np.isnan(pv)] = 1
    _, pv, _, _ = multipletests(pv, method='fdr_bh')

    df['corr_significance_log'] = -np.log10(pv)
    better_method = np.ones(df.shape[0])
    better_method[(df['SCARlink corr'] < df['ArchR gene corr']).values] = -1
    df['better_method_flag'] = better_method
    df['corr_significance_log'] = df['corr_significance_log']*df['better_method_flag']
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#E6A0C4", "white", "#7294D4"])
    cmap1 = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white", "#E6A0C4"])
    cmap2 = matplotlib.colors.LinearSegmentedColormap.from_list("", ["white", "#7294D4"])
    fig, ax = plt.subplots(1, 3, width_ratios=[1, 0.05, 0.05], figsize=(5.5, 3.5))
    ax[0].scatter(df['ArchR gene corr'], df['SCARlink corr'], c=df['corr_significance_log'], cmap=cmap, alpha=0.8, vmin=-4, vmax=4, edgecolors='white', linewidth=0.7)
    norm = plt.Normalize(-4, 4)
    norm2 = plt.Normalize(0, 4)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm) # "BrBG"
    sm1 = plt.cm.ScalarMappable(cmap=cmap1, norm=norm2) # "BrBG"
    sm2 = plt.cm.ScalarMappable(cmap=cmap2, norm=norm2) # "BrBG"
    sm.set_array([])
    sm1.set_array([])
    sm2.set_array([])
    fig.colorbar(sm1, cax=ax[1], orientation='vertical')
    fig.colorbar(sm2, cax=ax[2], orientation='vertical')
    ax[0].plot([min_val-0.01, max_val+0.01], [min_val-0.01, max_val+0.01], ls='--', color='black')
    ax[0].set_xlim((min_val-0.01, max_val+0.01))
    ax[0].set_ylim((min_val-0.01, max_val+0.01))
    ax[0].set_xlabel("ArchR GSM correlation")
    ax[0].set_ylabel("SCARlink correlation")
    plt.suptitle(' '.join(plot_title.split('_')))
    plt.tight_layout()
    plt.savefig(output_prefix + '_compare_gsm.pdf', transparent=True)
    plt.show()
    
    ###
    print("SCARlink better:", df[(df['SCARlink corr'] > df['ArchR gene corr']) & (pv < 0.05)].shape[0], df[(df['SCARlink corr'] > df['ArchR gene corr']) & (pv < 0.05)].shape[0]/df.shape[0], df.shape)
    print("ArchR better:", df[(df['SCARlink corr'] < df['ArchR gene corr']) & (pv < 0.05)].shape[0], df[(df['SCARlink corr'] < df['ArchR gene corr']) & (pv < 0.05)].shape[0]/df.shape[0], df.shape)
    plt.close()
    nan_vals = (~df['SCARlink corr'].isna()).values & (~df['ArchR gene corr'].isna()).values
    _, pv = stats.wilcoxon(df['SCARlink corr'].values[nan_vals], df['ArchR gene corr'].values[nan_vals], alternative='greater')
    print("Wilcoxon p-val:", pv)

In [11]:
# make tmp directory
os.makedirs("./tmp/", exist_ok=True)
os.makedirs("./scarlink_outs/", exist_ok=True)

In [45]:
df=make_scarlink_table_DiffTechs(dirname1='../mouse_brain_10X', dirname2='../mouse_brain_BGI', out_prefix='./corr', check_hvg=True)

In [49]:
df['diff']=df['method 2']-df['method 1']

In [50]:
df.better_method_flag.value_counts()

better_method_flag
 1    229
-1     80
Name: count, dtype: int64

In [56]:
df['diff']

0     -0.005808
1     -0.066427
2     -0.013940
3     -0.051390
4     -0.076000
         ...   
304   -0.131121
305   -0.131385
306   -0.076148
307    0.023071
308   -0.084262
Name: diff, Length: 309, dtype: float64

In [28]:
df1=make_scarlink_table(dirname='../mouse_brain_10X', check_hvg=True)
df2=make_scarlink_table(dirname='../mouse_brain_BGI', check_hvg=True)

In [23]:
df1['corr'].mean()

0.25787602520779057

In [22]:
df2['corr'].mean()

0.25235644153560394

In [58]:
df1.to_csv('./corr_values_10X.csv', sep='\t', index=None)
df2.to_csv('./corr_values_BGI.csv', sep='\t', index=None)